In [9]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.discrete.count_model import ZeroInflatedPoisson

# Generate sample data
np.random.seed(0)
n = 1000
X1 = np.random.randn(n)
X2 = np.random.randn(n)

# Generate response variable with excess zeros
lambda_ = np.exp(1 + 0.5 * X1 - 0.5 * X2)
prob_zero = 1 / (1 + np.exp(-(0.5 - X1 + 0.5 * X2)))
y = np.random.poisson(lambda_)
y[np.random.random(n) < prob_zero] = 0

# Create a DataFrame
df = pd.DataFrame({'y': y, 'X1': X1, 'X2': X2})

# Prepare the data for ZeroInflatedPoisson
exog = sm.add_constant(df[['X1', 'X2']])
exog_infl = sm.add_constant(df[['X1', 'X2']])

# Fit Zero-Inflated Poisson model
zip_model = ZeroInflatedPoisson(df['y'], exog, exog_infl=exog_infl)
zip_results = zip_model.fit()

# Print summary
print(zip_results.summary())

# Access coefficients
print("\nCount model coefficients:")
print(zip_results.params[:3])  # First 3 are for the count model
print("\nZero-inflation model coefficients:")
print(zip_results.params[3:])  # Last 3 are for the zero-inflation model

# Predict
new_data = pd.DataFrame({'X1': [0.5, -0.5], 'X2': [-0.5, 0.5]})
new_exog = sm.add_constant(new_data)
predicted = zip_results.predict(new_exog, exog_infl=new_exog)
print("\nPredicted counts for new data:")
print(predicted)

Optimization terminated successfully.
         Current function value: 1.279062
         Iterations: 18
         Function evaluations: 20
         Gradient evaluations: 20
                     ZeroInflatedPoisson Regression Results                    
Dep. Variable:                       y   No. Observations:                 1000
Model:             ZeroInflatedPoisson   Df Residuals:                      997
Method:                            MLE   Df Model:                            2
Date:                 Sun, 29 Sep 2024   Pseudo R-squ.:                  0.2666
Time:                         21:14:12   Log-Likelihood:                -1279.1
converged:                        True   LL-Null:                       -1744.1
Covariance Type:             nonrobust   LLR p-value:                1.085e-202
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
inflate_const     0.4253

In [11]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.discrete.count_model import ZeroInflatedNegativeBinomialP

# Generate sample data
np.random.seed(0)
n = 1000
X1 = np.random.randn(n)
X2 = np.random.randn(n)

# Generate response variable with excess zeros and overdispersion
lambda_ = np.exp(1 + 0.5 * X1 - 0.5 * X2)
prob_zero = 1 / (1 + np.exp(-(0.5 - X1 + 0.5 * X2)))
y = np.random.negative_binomial(n=5, p=5/(5+lambda_))  # Using NB parameterization
y[np.random.random(n) < prob_zero] = 0

# Create a DataFrame
df = pd.DataFrame({'y': y, 'X1': X1, 'X2': X2})

# Prepare the data for ZeroInflatedNegativeBinomialP
exog = sm.add_constant(df[['X1', 'X2']])
exog_infl = sm.add_constant(df[['X1', 'X2']])

# Fit Zero-Inflated Negative Binomial model
zinb_model = ZeroInflatedNegativeBinomialP(df['y'], exog, exog_infl=exog_infl)
zinb_results = zinb_model.fit()

# Print summary
print(zinb_results.summary())

# Access coefficients
print("\nCount model coefficients:")
print(zinb_results.params[:3])  # First 3 are for the count model
print("\nZero-inflation model coefficients:")
print(zinb_results.params[3:-1])  # Last 3 (excluding alpha) are for the zero-inflation model
print("\nOverdispersion parameter (alpha):")
print(zinb_results.params[-1])

# Predict
new_data = pd.DataFrame({'X1': [0.5, -0.5], 'X2': [-0.5, 0.5]})
new_exog = sm.add_constant(new_data)
predicted = zinb_results.predict(new_exog, exog_infl=new_exog)
print("\nPredicted counts for new data:")
print(predicted)

Optimization terminated successfully.
         Current function value: 1.316239
         Iterations: 30
         Function evaluations: 32
         Gradient evaluations: 32
                     ZeroInflatedNegativeBinomialP Regression Results                    
Dep. Variable:                                 y   No. Observations:                 1000
Model:             ZeroInflatedNegativeBinomialP   Df Residuals:                      997
Method:                                      MLE   Df Model:                            2
Date:                           Sun, 29 Sep 2024   Pseudo R-squ.:                  0.1407
Time:                                   21:14:52   Log-Likelihood:                -1316.2
converged:                                  True   LL-Null:                       -1531.8
Covariance Type:                       nonrobust   LLR p-value:                 2.386e-94
                    coef    std err          z      P>|z|      [0.025      0.975]
--------------------------

C:\Users\josep\AppData\Local\Temp\ipykernel_23564\2166287844.py:38: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(zinb_results.params[-1])


In [17]:
zinb_model.

<bound method DiscreteModel.cov_params_func_l1 of <statsmodels.discrete.count_model.ZeroInflatedNegativeBinomialP object at 0x000001AD90052390>>

In [21]:

# Function to calculate πi (zero-inflation probability)
def calc_pi(params_infl, exog_infl):
    linear_pred = np.dot(exog_infl, params_infl)
    return 1 / (1 + np.exp(-linear_pred))

# Function to calculate μi (mean of the count process)
def calc_mu(params_count, exog):
    return np.exp(np.dot(exog, params_count))

# Extract parameters
params_count = zinb_results.params[:3]  # First 3 are for the count model
params_infl = zinb_results.params[3:-1]  # Next 3 are for the zero-inflation model

# Calculate πi and μi for the original data
pi_i = calc_pi(params_infl, exog_infl)
mu_i = calc_mu(params_count, exog)

# Print some sample results
print("\nSample results for 5 observations:")
for i in range(100):
    print(f"Observation {i+1}:")
    print(f"  πi (zero-inflation probability): {pi_i[i]:.4f}")
    print(f"  μi (mean of count process): {mu_i[i]:.4f}")

# For new data
new_data = pd.DataFrame({'X1': [0.5, -0.5], 'X2': [-0.5, 0.5]})
new_exog = sm.add_constant(new_data)

pi_i_new = calc_pi(params_infl, new_exog)
mu_i_new = calc_mu(params_count, new_exog)

print("\nResults for new data:")
for i in range(len(new_data)):
    print(f"New observation {i+1}:")
    print(f"  πi (zero-inflation probability): {pi_i_new[i]:.4f}")
    print(f"  μi (mean of count process): {mu_i_new[i]:.4f}")


Sample results for 5 observations:
Observation 1:
  πi (zero-inflation probability): 0.8021
  μi (mean of count process): 0.3954
Observation 2:
  πi (zero-inflation probability): 0.6587
  μi (mean of count process): 1.6029
Observation 3:
  πi (zero-inflation probability): 0.8336
  μi (mean of count process): 0.5361
Observation 4:
  πi (zero-inflation probability): 0.8630
  μi (mean of count process): 0.2101
Observation 5:
  πi (zero-inflation probability): 0.8349
  μi (mean of count process): 0.3124
Observation 6:
  πi (zero-inflation probability): 0.6155
  μi (mean of count process): 4.2399
Observation 7:
  πi (zero-inflation probability): 0.7454
  μi (mean of count process): 0.8311
Observation 8:
  πi (zero-inflation probability): 0.8710
  μi (mean of count process): 0.8354
Observation 9:
  πi (zero-inflation probability): 0.7241
  μi (mean of count process): 1.7031
Observation 10:
  πi (zero-inflation probability): 0.7346
  μi (mean of count process): 1.2004
Observation 11:
  πi (z